In [737]:
import pandas as pd
import plotly.express as px
from datetime import datetime
import os

In [738]:
pd.set_option("display.max_colwidth", None)  # after this you can print any column length
pd.set_option("display.max_rows", 7)  # after this you can print any column length

## Load data 

In [742]:
data_dir = 'data'
motions_data_path = os.path.join(data_dir, 'motions_bp.csv')

if not os.path.exists(motions_data_path): 
    raise FileNotFoundError('Local data file not found. Please run the data collection notebook first.')

df = pd.read_csv(motions_data_path, index_col=0)
df.head()

Motion  \
0                                                                      This house prefers that progressive journalists focus their efforts on working within and reforming mainstream news platforms, as opposed to establishing their own platforms.   
1                                                                                                                                                                                                       This house regrets the Indian Premier League.   
2                                                                                                                                                    This house believes that breaking up large meat companies is in the interests of animal welfare.   
3                                                                                                                                       This house prefers a world where genius only develops after adulthood (so that there are no child prodigies).   
4  This house believes that East African national governments should prioritize investments in agriculture (e.g., irrigation, R&D, and inputs like fertilizer) over accelerating industrialization (e.g., building infrastructure, export promotion).   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     Info Slide  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           NaN   
1  The Indian Premier League (IPL) is a domestic cricket league in India. Since\nits inception in 2007, it has grown to become the largest cricket league in\nthe world, attracting sponsorship that exceeds that of most national teams.\n\nTraditionally, cricket competition has been dominated by international events\nand national teams touring other countries, and has involved games lasting\neither five days or one day. The IPL popularized large-scale domestic cricket\nleagues, and is played in a shorter, three-hour format of cricket called\nTwenty20 (T20). Teams are allowed up to 4 international (non-Indian) players\nin their teams. Additionally, the IPL usually lasts 3-4 months, taking up a\nsignificant portion of the annual cricket calendar.   
2                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

### Parse the embedding field as float[]

In [743]:
import json
df['Embedding'] = df['Embedding'].map(json.loads)

## Motion types

### All-time motion types

In [744]:
counts = df.value_counts('Motion Type')
print(counts)

px.bar(
    counts, 
    y='count', 
    title='Motion Types',
    labels={'Motion Type': 'Motion Type'},
).show()

Motion Type
believes    578
would       167
prefers      78
           ... 
hopes         2
predicts      2
welcomes      1
Name: count, Length: 10, dtype: int64


### This year's motion types

In [745]:
current_year = datetime.now().year

df_this_year = df[df['Year'] == current_year]
df_this_year.tail()

,Motion,Info Slide,Tournament,Year,Motion Type,Motion (cleaned),Embedding
873,This House believes that states with low fertility rates should implement the Hanson Proposal.,"The Hanson Proposal would provide parents with a portion of all tax revenue\npaid by their children. It would also provide them with tax credits each year\nuntil their child turns 18. This would be revenue neutral, funded by the\nabolition of aged pensions.",easters2025:easters2025,2025.0,believes,states with low fertility rates should implement the hanson proposal,"[-0.015229817479848862, -0.029109833762049675, 0.030638910830020905, 0.054513197392225266, 0.051238104701042175, -0.026804668828845024, 0.03273724764585495, 0.05864838510751724, 0.01058893371373415, -0.004195954650640488, -0.008671175688505173, -0.047203704714775085, -0.0281707514077425, -0.07280382513999939, -0.061972688883543015, 0.04559854418039322, -0.005855462979525328, -0.059341851621866226, -0.0455513559281826, -0.024923603981733322, 0.05287744477391243, -0.03561321273446083, -0.006607668474316597, -0.010876677930355072, -0.010567160323262215, -0.05274181813001633, -0.034761495888233185, 0.03392648324370384, 0.016192611306905746, 0.02258683368563652, -0.055618029087781906, 0.015595021657645702, 0.0026999025139957666, 0.0031816880218684673, 0.001167929614894092, 0.04411647468805313, -0.023062683641910553, 0.028971584513783455, 0.10562282055616379, 0.013520946726202965, -0.010350625030696392, 0.023804299533367157, -0.04098568111658096, -0.02175172045826912, 0.04004906862974167, 0.024571526795625687, 0.03943117707967758, 0.06506957113742828, 0.053254254162311554, -0.03108929470181465, 0.012084114365279675, 0.014935887418687344, 0.0015096226707100868, 0.057114750146865845, -0.020834553986787796, 0.045158371329307556, -0.029522281140089035, -0.017699360847473145, -0.011487252078950405, 0.030156301334500313, -0.019018620252609253, -0.048488494008779526, -0.050389084964990616, 0.018084347248077393, -0.0315985344350338, 0.028716986998915672, 0.08593173325061798, -0.01161234825849533, -0.00910922884941101, 0.02033184841275215, -0.026790989562869072, -0.004129417706280947, 0.04525717720389366, 0.008680235594511032, 0.022787820547819138, -0.057426583021879196, 0.026738325133919716, -0.009069696068763733, 0.023311680182814598, 0.03892507776618004, 0.01888754777610302, -0.054099760949611664, -0.02059652842581272, -0.02291230671107769, 0.0060868957079946995, 0.04463657736778259, -0.06866283714771271, -0.03586973249912262, -0.0054259835742414, 0.020582713186740875, 0.027757056057453156, -0.044520653784275055, 0.008618694730103016, -0.03474203124642372, 0.012270570732653141, 0.022132646292448044, 0.05781150981783867, 0.024059906601905823, -0.06531211733818054, -0.03317214921116829, ...]"
874,This House believes that we support the use of analogue listing rather than individual listing when prohibiting NPS.,"New psychoactive substances (NPS) are synthetic drugs designed to mimic the\neffects of illegal substances, with altered chemical structures to evade drug\nlaws. New versions of these drugs are continuously being produced by\nclandestine drug laboratories and released into the public, predominantly for\nrecreational use. \nThere are two approaches to prohibiting these drugs. Individual listing, which\nis traditionally used in drug listing, only prohibits drugs explicitly named\nby legislation. Analogue listing pre-emptively prohibits drugs that are\nsimilar to existing illegal substances, either based on chemical composition\nor effects of consumption.",easters2025:easters2025,2025.0,believes,we support the use of analogue listing rather than individual listing when prohibiting nps,"[-0.027738507837057114, -0.009852658025920391, -0.026600351557135582, -0.009451375342905521, 0.03526436164975166, 0.06878688186407089, 0.040458522737026215, 0.04323339834809303, 0.0026819214690476656, -0.012263728305697441, -0.014241297729313374, -0.018102403730154037, 0.012505503371357918, -0.034192316234111786, -0.04624

In [746]:
counts = df_this_year.value_counts('Motion Type')

px.bar(
    counts, 
    y='count', 
    title=f'Motion Types for {current_year}',
    labels={'Motion Type': 'Motion Type'},
).show()

## Semantic analysis

In [747]:
%pip install -q scikit-learn


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [748]:
# spread embeddings across multiple columns

df_embeddings = pd.DataFrame(df['Embedding'].to_list())
df_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.044743,-0.004066,0.059965,0.008645,-0.007011,0.011290,-0.004182,0.056300,0.002027,0.039398,...,0.059510,0.018012,0.050039,0.028682,-0.047626,-0.023985,0.020677,0.030342,-0.046558,-0.010739
1,0.007604,0.012494,-0.010231,0.002529,0.019429,0.065401,-0.011985,-0.034474,-0.031734,-0.021229,...,0.043262,0.044202,0.009047,0.037140,-0.017119,0.012782,-0.017478,0.004641,0.002280,-0.002080
2,-0.037876,-0.024296,0.036934,-0.061502,0.018667,0.023816,-0.025736,0.043228,-0.049594,0.013446,...,0.056465,-0.028549,-0.004228,0.029908,0.027661,-0.010363,-0.049724,0.012372,0.023746,-0.064294
3,-0.043584,0.023003,0.025124,0.033480,-0.011414,0.040876,0.023582,-0.001648,-0.017460,-0.003488,...,0.077739,-0.001549,0.025300,-0.060920,-0.004132,0.013044,0.013875,0.015683,0.038226,-0.016873
4,-0.006614,0.005584,0.031920,-0.015896,0.042803,0.057063,-0.013449,0.078428,-0.025080,0.016653,...,0.101879,-0.035559,0.017335,0.008178,-0.069794,-0.040476,0.054370,-0.008224,0.021047,-0.004360


### Standardize features

In [749]:
def check_distr(df: pd.DataFrame):
    mse_mean = ((df.mean() - 0) ** 2).mean()
    mse_std = ((df.std() - 1) ** 2).mean()

    print(f"Mean: {df.values.mean()}, mse: {mse_mean}")
    print(f"St.d.: {df.values.std()}, mse: {mse_std}")

check_distr(df_embeddings)

Mean: -0.0019674286263968357, mse: 0.00050930336547903
St.d.: 0.03603067275290286, mse: 0.9447109695885593


In [750]:
from sklearn.preprocessing import StandardScaler

scaled = StandardScaler().fit_transform(df_embeddings)
df_embeddings = pd.DataFrame(scaled)

check_distr(df_embeddings)

Mean: 3.4774542479192176e-19, mse: 2.1094787973374277e-33
St.d.: 0.9999999999999999, mse: 2.449527636078718e-07


### KMeans

In [751]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

silhouettes = []

k_range = (2, min(len(df_embeddings), 100))

def find_best_n_clusters(start: int = k_range[0], end: int = k_range[1]):
    best_n = dict(
        n=0.0,
        score=-1.0
    )

    for n_clusters in range(start, end):
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        kmeans.fit(df_embeddings)

        df_kmeans = pd.DataFrame(kmeans.labels_, columns=['centroid'])

        silhouette_avg = float(silhouette_score(df_embeddings, df_kmeans.values.ravel()))
        silhouettes.append(silhouette_avg)

        if silhouette_avg > best_n['score']:
            best_n['score'] = silhouette_avg
            best_n['n'] = n_clusters
    
    return best_n

best_n_clusters = find_best_n_clusters()

In [776]:
print(f"Best n clusters = {best_n_clusters['n']} with a silhouette score of {best_n_clusters['score']}")
fig = px.line(x=range(*k_range), y=silhouettes)

avg = sum(silhouettes) / len(silhouettes)
fig.add_hline(y=avg, line_dash='dash')

Best n clusters = 46 with a silhouette score of 0.03242653884594308


In [753]:
n_clusters = 21

kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(df_embeddings)

labels = kmeans.labels_
centroids = kmeans.cluster_centers_

df_kmeans = pd.DataFrame(labels, columns=['centroid'])
df_kmeans.head()

,centroid
0,6
1,18
2,2
3,1
4,10


In [754]:
# import matplotlib.pyplot as plt
# import matplotlib.cm as cm
# import numpy as np
# from sklearn.metrics import silhouette_samples

# sample_silhouette_values = silhouette_samples(df_embeddings, df_kmeans.values.ravel())

# ax1 = plt.subplot()

# y_lower = 10
# for i in range(n_clusters):
#     # Aggregate the silhouette scores for samples belonging to
#     # cluster i, and sort them
#     ith_cluster_silhouette_values = sample_silhouette_values[df_kmeans.values.ravel() == i]

#     ith_cluster_silhouette_values.sort()

#     size_cluster_i = ith_cluster_silhouette_values.shape[0]
#     y_upper = y_lower + size_cluster_i

#     color = cm.nipy_spectral(float(i) / n_clusters)
#     ax1.fill_betweenx(
#         np.arange(y_lower, y_upper),
#         0,
#         ith_cluster_silhouette_values,
#         facecolor=color,
#         edgecolor=color,
#         alpha=0.7,
#     )

#     # Label the silhouette plots with their cluster numbers at the middle
#     ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

#     # Compute the new y_lower for next plot
#     y_lower = y_upper + 10  # 10 for the 0 samples

# ax1.set_title("The silhouette plot for the various clusters.")
# ax1.set_xlabel("The silhouette coefficient values")
# ax1.set_ylabel("Cluster label")

# # The vertical line for average silhouette score of all the values
# ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

# ax1.set_yticks([])  # Clear the yaxis labels / ticks
# # ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

### PCA

In [755]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca_embeddings = pca.fit_transform(df_embeddings)
df_pca = pd.DataFrame(pca_embeddings, columns=['pca_1', 'pca_2'])

df_pca.head(), pca_embeddings.shape, pca.explained_variance_ratio_

(      pca_1     pca_2
 0  3.788042  0.876446
 1 -0.592633  3.339309
 2 -5.290854 -5.023173
 3  8.567714  0.725477
 4 -8.281304  6.501644,
 (1011, 2),
 array([0.03763042, 0.02826627]))

In [756]:
# centroids through PCA
pca_centroids = pca.transform(centroids)
pca_centroids.shape

(21, 2)

In [757]:
df_with_emb = pd.concat([df.drop(columns=['Embedding']), df_pca, df_kmeans], axis=1)
df_with_emb.head()

,Motion,Info Slide,Tournament,Year,Motion Type,Motion (cleaned),pca_1,pca_2,centroid
0,"This house prefers that progressive journalists focus their efforts on working within and reforming mainstream news platforms, as opposed to establishing their own platforms.",NaN,hwsrr2025:_,2025.0,prefers,"progressive journalists focus their efforts on working within and reforming mainstream news platforms, as opposed to establishing their own platforms",3.788042,0.876446,6
1,This house regrets the Indian Premier League.,"The Indian Premier League (IPL) is a domestic cricket league in India. Since\nits inception in 2007, it has grown to become the largest cricket league in\nthe world, attracting sponsorship that exceeds that of most national teams.\n\nTraditionally, cricket competition has been dominated by international events\nand national teams touring other countries, and has involved games lasting\neither five days or one day. The IPL popularized large-scale domestic cricket\nleagues, and is played in a shorter, three-hour format of cricket called\nTwenty20 (T20). Teams are allowed up to 4 international (non-Indian) players\nin their teams. Additionally, the IPL usually lasts 3-4 months, taking up a\nsignificant portion of the annual cricket calendar.",hwsrr2025:_,2025.0,regrets,indian premier league,-0.592633,3.339309,18
2,This house believes that breaking up large meat companies is in the interests of animal welfare.,NaN,hwsrr2025:_,2025.0,believes,breaking up large meat companies is in the interests of animal welfare,-5.290854,-5.023173,2
3,This house prefers a world where genius only develops after adulthood (so that there are no child prodigies).,NaN,hwsrr2025:_,2025.0,prefers,a world where genius only develops after adulthood (so that there are no child prodigies),8.567714,0.725477,1
4,"This house believes that East African national governments should prioritize investments in agriculture (e.g., irrigation, R&D, and inputs like fertilizer) over accelerating industrialization (e.g., building infrastructure, export promotion).",NaN,hwsrr2025:_,2025.0,believes,"east african national governments should prioritize investments in agriculture (eg, irrigation, r&d, and inputs like fertilizer) over accelerating industrialization (eg, building infrastructure, export promotion)",-8.281304,6.501644,10


In [758]:
import plotly.graph_objects as go

fig = go.Figure()

marker = go.scatter.Marker(
    color=df_with_emb['centroid'],
    colorscale='rainbow',
    size=10,
    opacity=0.7
)

centroid_marker = go.scatter.Marker(
    symbol='x',
    size=20,
    opacity=0.5,
    color='gray'
)

fig.add_trace(go.Scatter(x=df_with_emb['pca_1'], y=df_with_emb['pca_2'], mode='markers', hovertext=df_with_emb['Motion'], marker=marker))
# fig.add_trace(go.Scatter(x=pca_centroids[:, 0], y=pca_centroids[:, 1], mode='markers', marker=centroid_marker))

fig.show()

In [759]:
df_with_emb.sort_values('centroid')[['Motion (cleaned)', 'centroid']]

,Motion (cleaned),centroid
209,technocrats should govern the state for the first five years,0
968,democracies should hold elections immediately post-covid,0
392,leaders of an occupied nation should not assist the occupying powers in maintaining stability,0
...,...,...
978,we should abolish academic tenure,20
570,"impose a cap on university endowments, in favour of directing the money towards grants given to individual professors/researchers",20
190,education systems in africa should emphasise pan-african history and identities rather than national ones,20


### t-SNE

In [760]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=3, random_state=42)
X_tsne = tsne.fit_transform(df_embeddings)

df_tsne = pd.DataFrame(X_tsne, columns=['tsne_1', 'tsne_2', 'tsne_3'])
tsne.kl_divergence_

1.4702579975128174

In [761]:
df_with_emb2 = pd.concat([df_with_emb, df_tsne], axis=1)

In [773]:
px.scatter_3d(df_with_emb2, x='tsne_1', y='tsne_2', z='tsne_3', hover_data='Motion', color='centroid',color_continuous_scale=px.colors.sequential.Jet)

## Team wins (balance)

In [713]:
prop_wins = df['Prop wins'].sum()
opp_wins = df['Opp wins'].sum()

print(prop_wins, opp_wins)

px.pie(
    values=[prop_wins, opp_wins],
    names=['Prop wins', 'Opp wins'],
    title='Total Prop vs Opp Wins',
).show()

KeyError: 'Prop wins'

In [11]:
prop_wins = df_this_year['Prop wins'].sum()
opp_wins = df_this_year['Opp wins'].sum()

print(prop_wins, opp_wins)

px.pie(
    values=[prop_wins, opp_wins],
    names=['Prop wins', 'Opp wins'],
    title='Total Prop vs Opp Wins',
).show()

196.0 85.0
